In [1]:
!pip install tokenizers
!pip install torchtext
!pip install pytorch_lightning
!pip install datasets
!pip install tensorboard

You should consider upgrading via the '/Users/shreyash/Developer/TSAI/ERAV2/erav2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shreyash/Developer/TSAI/ERAV2/erav2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shreyash/Developer/TSAI/ERAV2/erav2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shreyash/Developer/TSAI/ERAV2/erav2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shreyash/Developer/TSAI/ERAV2/erav2/bin/python3 -m pip install --upgrade pip' command.


In [16]:
# import torch
# import torch.nn as nn
# import math

# class LayerNormalization(nn.Module):

#     def __init__(self, eps:float=10**-6) -> None:
#         super().__init__()
#         self.eps = eps
#         self.alpha = nn.Parameter(torch.ones(1)) # alpha is a learnable parameter
#         self.bias = nn.Parameter(torch.zeros(1)) # bias is a learnable parameter

#     def forward(self, x):
#         # x: (batch, seq_len, hidden_size)
#         # Keep the dimension for broadcasting
#         mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
#         # Keep the dimension for broadcasting
#         std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
#         # eps is to prevent dividing by zero or when std is very small
#         return self.alpha * (x - mean) / (std + self.eps) + self.bias

# class FeedForwardBlock(nn.Module):

#     def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
#         super().__init__()
#         self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
#         self.dropout = nn.Dropout(dropout)
#         self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

#     def forward(self, x):
#         # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
#         return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

# class InputEmbeddings(nn.Module):

#     def __init__(self, d_model: int, vocab_size: int) -> None:
#         super().__init__()
#         self.d_model = d_model
#         self.vocab_size = vocab_size
#         self.embedding = nn.Embedding(vocab_size, d_model)

#     def forward(self, x):
#         # (batch, seq_len) --> (batch, seq_len, d_model)
#         # Multiply by sqrt(d_model) to scale the embeddings according to the paper
#         return self.embedding(x) * math.sqrt(self.d_model)
    
# class PositionalEncoding(nn.Module):

#     def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
#         super().__init__()
#         self.d_model = d_model
#         self.seq_len = seq_len
#         self.dropout = nn.Dropout(dropout)
#         # Create a matrix of shape (seq_len, d_model)
#         pe = torch.zeros(seq_len, d_model)
#         # Create a vector of shape (seq_len)
#         position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
#         # Create a vector of shape (d_model)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
#         # Apply sine to even indices
#         pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
#         # Apply cosine to odd indices
#         pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
#         # Add a batch dimension to the positional encoding
#         pe = pe.unsqueeze(0) # (1, seq_len, d_model)
#         # Register the positional encoding as a buffer
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
#         return self.dropout(x)

# class ResidualConnection(nn.Module):
    
#         def __init__(self, dropout: float) -> None:
#             super().__init__()
#             self.dropout = nn.Dropout(dropout)
#             self.norm = LayerNormalization()
    
#         def forward(self, x, sublayer):
#             return x + self.dropout(sublayer(self.norm(x)))

# class MultiHeadAttentionBlock(nn.Module):

#     def __init__(self, d_model: int, h: int, dropout: float) -> None:
#         super().__init__()
#         self.d_model = d_model # Embedding vector size
#         self.h = h # Number of heads
#         # Make sure d_model is divisible by h
#         assert d_model % h == 0, "d_model is not divisible by h"

#         self.d_k = d_model // h # Dimension of vector seen by each head
#         self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
#         self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
#         self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
#         self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
#         self.dropout = nn.Dropout(dropout)

#     @staticmethod
#     def attention(query, key, value, mask, dropout: nn.Dropout):
#         d_k = query.shape[-1]
#         # Just apply the formula from the paper
#         # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
#         attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
#         #S16
#         _MASKING_VALUE=-1e30 if attention_scores.dtype==torch.float32 else -1e4
#         if mask is not None:
#             # Write a very low value (indicating -inf) to the positions where mask == 0
#             # attention_scores.masked_fill_(mask == 0, -1e9)
#             # print('attetion score matrix size',attention_scores.shape, "mask shape",  mask.shape)
#             attention_scores.masked_fill_(mask==0, value=_MASKING_VALUE)
#         attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
#         if dropout is not None:
#             attention_scores = dropout(attention_scores)
#         # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
#         # return attention scores which can be used for visualization
#         return (attention_scores @ value), attention_scores

#     def forward(self, q, k, v, mask):
#         query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
#         key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
#         value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

#         # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
#         query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
#         key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
#         value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

#         # Calculate attention
#         x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
#         # Combine all the heads together
#         # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
#         x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

#         # Multiply by Wo
#         # (batch, seq_len, d_model) --> (batch, seq_len, d_model)  
#         return self.w_o(x)

# class EncoderBlock(nn.Module):

#     def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
#         super().__init__()
#         self.self_attention_block = self_attention_block
#         self.feed_forward_block = feed_forward_block
#         self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

#     def forward(self, x, src_mask):
#         x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
#         x = self.residual_connections[1](x, self.feed_forward_block)
#         return x
    
# class Encoder(nn.Module):

#     def __init__(self, layers: nn.ModuleList) -> None:
#         super().__init__()
#         self.layers = layers
#         self.norm = LayerNormalization()

#     def forward(self, x, mask):
#         for layer in self.layers:
#             x = layer(x, mask)
#         return self.norm(x)

# class DecoderBlock(nn.Module):

#     def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
#         super().__init__()
#         self.self_attention_block = self_attention_block
#         self.cross_attention_block = cross_attention_block
#         self.feed_forward_block = feed_forward_block
#         self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

#     def forward(self, x, encoder_output, src_mask, tgt_mask):
#         x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
#         x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
#         x = self.residual_connections[2](x, self.feed_forward_block)
#         return x
    
# class Decoder(nn.Module):

#     def __init__(self, layers: nn.ModuleList) -> None:
#         super().__init__()
#         self.layers = layers
#         self.norm = LayerNormalization()

#     def forward(self, x, encoder_output, src_mask, tgt_mask):
#         for layer in self.layers:
#             x = layer(x, encoder_output, src_mask, tgt_mask)
#         return self.norm(x)

# class ProjectionLayer(nn.Module):

#     def __init__(self, d_model, vocab_size) -> None:
#         super().__init__()
#         self.proj = nn.Linear(d_model, vocab_size)

#     def forward(self, x) -> None:
#         # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
#         return torch.log_softmax(self.proj(x), dim = -1)
    
# class Transformer(nn.Module):

#     def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
#         super().__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#         self.src_embed = src_embed
#         self.tgt_embed = tgt_embed
#         self.src_pos = src_pos
#         self.tgt_pos = tgt_pos
#         self.projection_layer = projection_layer

#     def encode(self, src, src_mask):
#         # (batch, seq_len, d_model)
#         src = self.src_embed(src)
#         src = self.src_pos(src)
#         return self.encoder(src, src_mask)
    
#     def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
#         # (batch, seq_len, d_model)
#         tgt = self.tgt_embed(tgt)
#         tgt = self.tgt_pos(tgt)
#         return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
#     def project(self, x):
#         # (batch, seq_len, vocab_size)
#         return self.projection_layer(x)
    
# def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
#     # Create the embedding layers
#     src_embed = InputEmbeddings(d_model, src_vocab_size)
#     tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

#     # Create the positional encoding layers
#     src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
#     tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    
#     # Create the encoder blocks
#     encoder_blocks = []
#     for _ in range(N):
#         encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
#         feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
#         encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
#         encoder_blocks.append(encoder_block)

#     # Create the decoder blocks
#     decoder_blocks = []
#     for _ in range(N):
#         decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
#         decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
#         feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
#         decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
#         decoder_blocks.append(decoder_block)
    
#     # Create the encoder and decoder
#     encoder = Encoder(nn.ModuleList(encoder_blocks))
#     decoder = Decoder(nn.ModuleList(decoder_blocks))
    
#     # Create the projection layer
#     projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    
#     # Create the transformer
#     transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    
#     # Initialize the parameters
#     for p in transformer.parameters():
#         if p.dim() > 1:
#             nn.init.xavier_uniform_(p)
    
#     return transformer
# from dataset import BillingualDataset, casual_mask
# from config_file import get_config, get_weights_file_path

# import torchtext.datasets as datasets
# import torch
# torch.cuda.amp.autocast(enabled = True)

# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader, random_split
# from torch.optim.lr_scheduler import LambdaLR

# import warnings
# from tqdm import tqdm
# import os
# from pathlib import Path

# from datasets import load_dataset
# from tokenizers import Tokenizer
# from tokenizers.models import WordLevel
# from tokenizers.trainers import WordLevelTrainer
# from tokenizers.pre_tokenizers import Whitespace

# import torchmetrics
# from torch.utils.tensorboard import SummaryWriter

# torch.cuda.empty_cache()
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:12240"
# config = get_config()

# def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    
    
#     sos_idx = tokenizer_tgt.token_to_id('[SOS]')
#     eos_idx = tokenizer_tgt.token_to_id('[EOS]')
    
#     encoder_output = model.encode(source, source_mask)
#     #Initialize the decoder input with SOS token
#     decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
#     while True:
#         if decoder_input.size(1) == max_len:
#             break
#         decoder_mask = casual_mask(decoder_input.size(1)).type_as(source_mask).to(device)
#         out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        
#         prob = model.project(out[:, -1])
#         _, next_word = torch.max(prob, dim=1)
#         decoder_input = torch.cat(
#             [
#                 decoder_input,
#                 torch.empty(1, 1).type_as(source_mask).fill_(next_word.item()).to(device)
#             ],
#             dim =  1
#         )
        
#         if next_word == eos_idx:
#             break
        
#     return decoder_input.squeeze(0)


# def run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, max_len, device, writer, global_step):
#     model.eval()
#     count = 0
#     source_texts = []
#     expected = []
#     predicted = []
    
#     try:
#         with os.popen('stty size', 'r') as console:
#             _, console_width = console.read().split()
#             console_width = int(console_width)
#     except:
#         console_width = 80
        
#     with torch.no_grad():
#         for batch in val_dataloader:
#             count += 1
#             encoder_input = batch["encoder_input"].to(device)
#             encoder_mask = batch["encoder_mask"].to(device)
            
#             assert encoder_input.size(0)==1, "Batch size must be 1 for validation"
#             model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)
            
#             source_text = batch["src_text"][0]
#             target_text = batch["tgt_text"][0]
#             model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())
            
#             source_texts.append(source_text)
#             expected.append(target_text)
#             predicted.append(model_out_text)
#     """        
#             print("SOURCE", source_text)
#             print("TARGET", target_text)
#             print("PREDICTED", model_out_text)
            
#     if writer:
#         metric = torchmetrics.CharErrorRate()
#         cer = metric(predicted, expected)
#         writer.add_scalar('validation cer', cer, global_step)
#         writer.flush()
        
#         metric = torchmetrics.WordErrorRate()
#         wer = metric(predicted, expected)
#         writer.add_scalar('validation wer', wer, global_step)
#         writer.flush()
        
#         metric = torchmetrics.BLEUScore()
#         bleu = metric(predicted, expected)
#         writer.add_scalar('validation BLEU', bleu, global_step)
#         writer.flush()
        
#     """   

# def get_all_sentenses(ds, lang):
#     for item in ds:
#         yield item['translation'][lang]
        
# def get_or_build_tokenizer(config, ds, lang):
#     tokenizer_path = Path(config["tokenizer_file"].format(lang))
#     if not Path.exists(tokenizer_path):
#         tokenizer = Tokenizer(WordLevel(unk_token = "[UNK]"))
#         tokenizer.pre_tokenizer = Whitespace() 
#         trainer = WordLevelTrainer(special_tokens = ["[UNK]", "[SOS]", "[EOS]", "[PAD]"], min_frequency = 2)
#         tokenizer.train_from_iterator(get_all_sentenses(ds, lang), trainer = trainer)
#         tokenizer.save(str(tokenizer_path))
#     else:
#         tokenizer = Tokenizer.from_file(str(tokenizer_path))

#     return tokenizer


# def get_ds(config):
    
#     ds_raw = load_dataset('opus_books', f"{config['lang_src']}-{config['lang_tgt']}", split = 'train')  
    
#     src_lang = config["lang_src"]
#     tgt_lang = config["lang_tgt"]
#     seq_len = config["seq_len"]
    
#     # print(ds_raw[0], type(ds_raw), type(ds_raw[0]))
    
#     tokenizer_src = get_or_build_tokenizer(config, ds_raw, src_lang)
#     tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, tgt_lang)
    
#     # ds_raw = sorted(ds_raw, key=lambda x: len(tokenizer_src.encode(x['translation'][src_lang]).ids))
#     # print(ds_raw[0], type(ds_raw), type(ds_raw[0]))
    
#     train_ds_size = int(0.9 * len(ds_raw))
#     val_ds_size = len(ds_raw) - train_ds_size
#     train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])
    
#     # print(type(ds_raw), train_ds_raw[0], type(train_ds_raw), type(train_ds_raw[0]))
    
#     # train_ds_list = []
    
#     # for item in train_ds_raw:
#     #     train_ds_list.append((src_ids, tgt_ids))
    
#     # train_ds_raw.sort(key=lambda x: len(tokenizer_src.encode(x['translation'][src_lang]).ids))
    
    

#     train_ds = BillingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len)
#     val_ds = BillingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len)
    
#     # print(train_ds[0], train_ds, type(train_ds), type(train_ds[0]))
    
#     max_len_src = 0
#     max_len_tgt = 0
    
#     for item in ds_raw:
#         src_ids = tokenizer_src.encode(item['translation'][src_lang]).ids
#         tgt_ids = tokenizer_tgt.encode(item['translation'][tgt_lang]).ids
#         max_len_src = max(max_len_src, len(src_ids))
#         max_len_tgt = max(max_len_tgt, len(tgt_ids))
    
#     print(f"Max length of the source sentence : {max_len_src}")
#     print(f"Max length of the source target : {max_len_tgt}")
    
#     train_dataloader = DataLoader(train_ds, batch_size = config["batch_size"], shuffle = True,  collate_fn=collate_func)
#     # print("Train dataloader created", next(iter(train_dataloader)))
#     val_dataloader = DataLoader(val_ds, batch_size = 1, shuffle = True)
    
#     return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

# def collate_func(batch):
#     max_encoder_len = 0
#     max_decoder_len = 0
#     for b  in batch:
#         max_encoder_len = max(max_encoder_len, len(b["encoder_input_without_padding"]))
#         max_decoder_len = max(max_decoder_len, len(b["decoder_input_without_padding"] ))
    
#     seq_len_this_batch = max(max_encoder_len, max_decoder_len) + 3
    
#     # print(max_encoder_len, max_decoder_len, seq_len_this_batch)
    
#     for b in batch:
#         enc_num_padding_tokens = seq_len_this_batch - len(b["encoder_input_without_padding"] )
#         dec_num_padding_tokens = seq_len_this_batch - len(b["decoder_input_without_padding"] )
#         # print(enc_num_padding_tokens, dec_num_padding_tokens)
#         # print(b["encoder_input_without_padding"])
#         # print(b["decoder_input_without_padding"])
#         encoder_input = torch.cat(
#             [
#                 b["encoder_input_without_padding"],
#                 torch.tensor([b["pad_token"]]*enc_num_padding_tokens, dtype = torch.int64)
#             ],
#             dim=0,
            
#         )
#         decoder_input = torch.cat(
#             [
#                 b["decoder_input_without_padding"],
#                 torch.tensor([b["pad_token"]]*dec_num_padding_tokens, dtype = torch.int64)
#             ],
#             dim=0
#         )
#         label = torch.cat(
#             [  
#                 b["label_without_padding"],
#                 torch.tensor([b["pad_token"]]*dec_num_padding_tokens, dtype = torch.int64)
#             ],
#             dim=0
#         )
#         b["encoder_input"] = encoder_input
#         b["decoder_input"] = decoder_input
#         b["label"] = label
#         b["encoder_mask"] =  ((encoder_input != b["pad_token"]).unsqueeze(0)).unsqueeze(0).unsqueeze(0).int()
#         # print(b["encoder_mask"].shape)
#         b["decoder_mask"] = (decoder_input != b["pad_token"]).unsqueeze(0).int() & casual_mask(decoder_input.size(0)).unsqueeze(0)
#         # print(b["encoder_input"])
#         # print(b["decoder_input"])
#         # print(b["label"])
#         # print(b["encoder_mask"])
#         # print(b["decoder_mask"])
#         # print(b["src_text"])
#         # print(b["tgt_text"])
#         # print(" ")
    
#     encoder_inputs = []
#     decoder_inputs = []
#     encoder_masks = []
#     decoder_masks = []
#     labels = []
#     src_texts = []
#     tgt_texts = []
    
#     for b in batch:
#         encoder_inputs.append(b["encoder_input"])
#         decoder_inputs.append(b["decoder_input"])
#         encoder_masks.append(b["encoder_mask"])
#         decoder_masks.append(b["decoder_mask"])
#         labels.append(b["label"])
#         src_texts.append(b["src_text"])
#         tgt_texts.append(b["tgt_text"])
        
#     # print(encoder_inputs)
    
    
#     # for b in batch:
        
        
    
#     return {
#         "encoder_input": torch.vstack(encoder_inputs),
#         "decoder_input": torch.vstack(decoder_inputs),
#         "encoder_mask": torch.vstack(encoder_masks),
#         "decoder_mask": torch.vstack(decoder_masks),
#         "label": torch.vstack(labels),
#         "src_text": src_texts,
#         "tgt_text": tgt_texts
#     }
    
    

# def get_model(config, src_vocab_size, tgt_vocab_size):
#     model = build_transformer(src_vocab_size, tgt_vocab_size, config["seq_len"], config["seq_len"], d_model=config['d_model'])
#     return model


# def train_model(config):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     device = torch.device("mps" if torch.backends.mps.is_available() else device)
#     print(f"Using device : {device}")
    
#     Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)
#     train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
#     model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
    
#     #Tensorboard
#     writer = SummaryWriter(config["experiment_name"])
    
#     #Adam is used to train each feature with a different learning rate. 
#     #If some feature is appearing less, adam takes care of it
#     optimizer = torch.optim.Adam(model.parameters(), lr = config["lr"], eps = 1e-9)
    
#     initial_epoch = 0
#     global_step = 0
    
#     if config["preload"]:
#         model_filename = get_weights_file_path(config, config["preload"])
#         print("Preloading model {model_filename}")
#         state = torch.load(model_filename)
#         model.load_state_dict(state["model_state_dict"])
#         initial_epoch = state["epoch"] + 1
#         optimizer.load_state_dict(state["optimizer_state_dict"])
#         global_step = state["global_step"]
#         print("preloaded")
        
#     loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1)
    
#     for epoch in range(initial_epoch, config["num_epochs"]):
#         torch.cuda.empty_cache()
#         print(epoch)
#         model.train()
#         batch_iterator = tqdm(train_dataloader, desc = f"Processing Epoch {epoch:02d}")
        
#         for batch in batch_iterator:
#             encoder_input = batch["encoder_input"].to(device)
#             decoder_input = batch["decoder_input"].to(device)
#             encoder_mask = batch["encoder_mask"].to(device)
#             decoder_mask = batch["decoder_mask"].to(device)
            
#             encoder_output = model.encode(encoder_input, encoder_mask)
#             decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
#             proj_output = model.project(decoder_output)
            
#             label = batch["label"].to(device)
            
#             #Compute loss using cross entropy
#             tgt_vocab_size = tokenizer_tgt.get_vocab_size()
#             loss = loss_fn(proj_output.view(-1, tgt_vocab_size), label.view(-1))
#             batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

#             #Log the loss
#             writer.add_scalar('train_loss', loss.item(), global_step)
#             writer.flush()
            
#             #Backpropogate loss
#             loss.backward()
            
#             #Update weights
#             optimizer.step()
#             optimizer.zero_grad(set_to_none=True)
#             global_step+=1
            
#         #run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, writer, global_step)
        
        
#         model_filename = get_weights_file_path(config, f"{epoch:02d}")
#         torch.save(
#             {
#                 "epoch": epoch,
#                 "model_state_dict": model.state_dict(),
#                 "optimizer_state_dict": optimizer.state_dict(),
#                 "global_step": global_step
#             },
#             model_filename
#         )
        


/Users/shreyash/Developer/TSAI/ERAV2/erav2/lib/python3.8/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [19]:
from train import train_model
from config_file import get_config

config = get_config()
config["batch_size"] = 16
config["preload"] = None
config["num_epochs"] = 18
train_model(config)

/Users/shreyash/Developer/TSAI/ERAV2/erav2/lib/python3.8/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Using device : mps
Max length of the source sentence : 309
Max length of the source target : 274
0


Processing Epoch 00:   0%|          | 4/1819 [00:17<2:15:13,  4.47s/it, loss=9.585] 


RuntimeError: MPS backend out of memory (MPS allocated: 12.07 GB, other allocations: 6.04 GB, max allowed: 18.13 GB). Tried to allocate 253.64 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).